生成模拟样本和对应标签

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [23]:
fs = 48000
def generate_normal_signal(num_samples=1000, signal_length=2048, noise_level=0.1):
    # 生成模拟正常信号
    t = np.linspace(0, 1, fs)

    frequency = 29.95  
    amplitude = 1.0  
    normal_signal = amplitude * np.sin(2 * np.pi * frequency * t)

    noise = noise_level * np.random.normal(size=normal_signal.shape)
    normal_signal += noise

    return normal_signal[0:signal_length]

def generate_fault_signal(fault_frequency, num_samples=1000, signal_length=2048, noise_level=0.1):
    # 生成模拟故障信号
    t = np.linspace(0, 1, fs)

    normal_signal = generate_normal_signal(num_samples, signal_length, noise_level)
    fault_amplitude = 0.8
    fault_signal = fault_amplitude * np.sin(2 * np.pi * fault_frequency * t)
    fault_signal = fault_signal[0:signal_length]

    combined_signal = normal_signal + fault_signal

    return combined_signal

def generate_labels(num_samples, fault_type):

    if fault_type == 'normal':
        return np.zeros(num_samples)  # 正常信号标签为0
    elif fault_type == 'outer':
        return np.ones(num_samples)  # 外圈故障标签为1
    elif fault_type == 'inner':
        return np.full(num_samples, 2)  # 内圈故障标签为2
    elif fault_type == 'ball':
        return np.full(num_samples, 3)  # 球故障标签为3
    else:
        raise ValueError("Unknown fault type!")


In [24]:
num_signals = 30
signal_length = 2048

#生成正常信号
normal_signals = np.array([generate_normal_signal(num_samples=1000, signal_length=signal_length) for _ in range(num_signals)])
normal_labels = generate_labels(num_signals, 'normal')

#生成外圈故障信号
outer_fault_signals = np.array([generate_fault_signal(107.22) for _ in range(num_signals)])
outer_fault_labels = generate_labels(num_signals, 'outer')

#生成内圈故障信号
inner_fault_signals = np.array([generate_fault_signal(162.3) for _ in range(num_signals)])
inner_fault_labels = generate_labels(num_signals, 'inner')

#生成滚珠故障信号
ball_fault_signals = np.array([generate_fault_signal(70.68) for _ in range(num_signals)])
ball_fault_labels = generate_labels(num_signals, 'ball')

all_signals = np.concatenate([normal_signals, outer_fault_signals, inner_fault_signals, ball_fault_signals])
all_labels = np.concatenate([normal_labels, outer_fault_labels, inner_fault_labels, ball_fault_labels])

np.save('simulate_signals.npy', all_signals)
np.save('simulate_labels.npy', all_labels)

获取真实正常样本

In [25]:
#读取真实正常数据样本
source = loadmat('N.mat')

def get_data(source, fs):
    data_DE = []
    data_FE = []
    data_Speed = []

    for i, key in enumerate(source.keys()):
        if i == 3:
            data_DE = source[key].flatten()
        elif i == 4:
            data_FE = source[key].flatten()
        elif i == 5:
            data_Speed = source[key].flatten()
    t  = np.arange(len(data_DE))/fs
    return t, data_DE, data_FE, data_Speed

fs = 48000

t, data_DE, data_FE, data_Speed = get_data(source, fs)

#从数据中随机获取正常数据样本
def random_sample(long_array, sample_num):
    array_length = len(long_array)
    num_samples = sample_num
    samples = []
    if num_samples * 2048 > array_length:
        return samples
    start_indices = np.random.randint(0, array_length - 2047, num_samples)
    
    for start in start_indices:
        end = start + 2048
        if end > array_length:
            sample = long_array[-2048:]
            #print(len(sample))
        else:
            sample = long_array[start:end]
            #print(len(sample))
        samples.append(sample)
    return samples

real_samples = random_sample(data_DE, 30)
real_labels = generate_labels(30, 'normal')

np.save('real_signals.npy', real_samples)
np.save('real_labels.npy', real_labels)